https://gyro-interp.readthedocs.io/en/latest/examples.html

In [1]:
import pandas as pd

# see youth_indicators_tois.ipynb
data = pd.read_csv('../data/youth_indicators_tois.csv')
data

,TOI,bv,bv_err,li,prot,rhk,teff
0,101.01,0.656000,0.1,NaN,1.430321,NaN,5648.642857
1,102.01,0.507667,0.1,NaN,4.410011,NaN,6257.270417
2,103.01,0.291000,0.1,NaN,3.554638,NaN,6086.159167
3,104.01,0.545667,0.1,NaN,4.090360,NaN,6046.516250
4,105.01,0.687667,0.1,NaN,2.187803,NaN,5741.285333
...,...,...,...,...,...,...,...
4312,5023.01,NaN,NaN,NaN,NaN,NaN,3720.494840
4313,5024.01,0.612000,NaN,NaN,NaN,NaN,5985.393333
4314,5025.01,0.313000,NaN,NaN,NaN,NaN,8132.166667
4315,5026.01,0.700000,NaN,NaN,NaN,NaN,6444.962144


In [2]:
d=data.query("TOI==200.01").squeeze()
d

TOI        200.010000
bv           0.735647
bv_err       0.048146
li         108.107498
prot         5.475000
rhk         -4.090000
teff      5482.444615
Name: 98, dtype: float64

In [3]:
Prot, Prot_err = d.prot, 0.1
Teff, Teff_err = d.teff, 100

## gyro-interp

In [4]:
import numpy as np
from gyrointerp import gyro_age_posterior

# uniformly spaced grid between 0 and 2600 megayears
age_grid = np.linspace(0, 2600, 500)

# calculate the age posterior - takes ~30 seconds
age_posterior = gyro_age_posterior(
  Prot, Teff, Prot_err=Prot_err, Teff_err=Teff_err, age_grid=age_grid
)

# calculate dictionary of summary statistics
from gyrointerp import get_summary_statistics
result = get_summary_statistics(age_grid, age_posterior)

print(f"Age = {result['median']} +{result['+1sigma']} -{result['-1sigma']} Myr.")

Age = 221.13 +101.35 -95.34 Myr.


## age of young cluster planet candidates

In [12]:
import pandas as pd

loc = '~/github/research/project/young_ttvs/code/tois_yuanzhe'
candidates = pd.read_csv(f'{loc}/tois_in_cluster_candidates_yuanzhe.txt', delimiter='\t')
candidates.shape

(70, 3)

In [96]:
from tqdm import tqdm

age_grid = np.linspace(0, 2600, 500)

for i,row in tqdm(candidates.iterrows()):
    # print(row.TOI)
    toi = row.TOI
    d=data.query("TOI==@toi").squeeze()
    if len(d)>0:
        Prot, Prot_err = d.prot, 0.1
        Teff, Teff_err = d.teff, 100
        if (str(Prot)!='nan') and (str(Teff)!='nan'):
            print(f"Prot={Prot} d, Teff={Teff} K")
            
            # calculate the age posterior - takes ~30 seconds
            age_posterior = gyro_age_posterior(
              Prot, Teff, Prot_err=Prot_err, Teff_err=Teff_err, age_grid=age_grid
            )
            
            # calculate dictionary of summary statistics
            result = get_summary_statistics(age_grid, age_posterior)
            
            print(f"TOI-{toi}: Age = {result['median']} +{result['+1sigma']} -{result['-1sigma']} Myr.")
            break

54it [00:00, 325.55it/s]

Prot=2.925198 d, Teff=7137.558 K
TOI-2538.01: Age = nan +nan -nan Myr.


## custom

In [3]:
from gyrointerp.plotting import plot_prot_vs_teff

# write the results to the current working directory
outdir = "./"

# show these cluster Prot vs Teff datasets
reference_clusters = [
    'α Per', 'Pleiades', 'Blanco-1', 'Psc-Eri', 'NGC-3532', 'Group-X',
    'Praesepe', 'NGC-6811'
]

# underplot these polynomial fits
model_ids = [
    'α Per', '120-Myr', '300-Myr', 'Praesepe', 'NGC-6811'
]

# overplot these stars with big markers
custom_stardict = {
    "Kepler-1643": {"Prot":5.1, "Teff":4916, "m":"s", "c":"red"},
    "TOI-1136": {"Prot":8.7, "Teff":5770, "m":"X", "c":"pink"},
    "TOI-1937 A": {"Prot":6.6, "Teff":5798, "m":"P", "c":"aqua"},
}

# make the plot
plot_prot_vs_teff(
    outdir, reference_clusters=reference_clusters, model_ids=model_ids,
    custom_stardict=custom_stardict, writepdf=0
)

[W 240827 19:19:03 getters:72] No module named 'cdips'
[I 240827 19:19:03 getters:1551] Found /mnt_ut3/raid_ut3/home/jerome/github/research/project/ext_tools/gyro-interp/gyrointerp/data/interim/alpha-per/Boyle_table3_full_DR3_supp.csv; returning.
[I 240827 19:19:03 getters:1253] Found /mnt_ut3/raid_ut3/home/jerome/github/research/project/ext_tools/gyro-interp/gyrointerp/data/interim/psc-eri/Curtis_2019_X_GDR3_supplemented.csv, and not overwrite; returning.
[I 240827 19:19:03 getters:528] Found /mnt_ut3/raid_ut3/home/jerome/github/research/project/ext_tools/gyro-interp/gyrointerp/data/interim/blanco1/Gillen_2020_X_GDR3_supplemented.csv, and not overwrite; returning.
[I 240827 19:19:03 getters:663] Found /mnt_ut3/raid_ut3/home/jerome/github/research/project/ext_tools/gyro-interp/gyrointerp/data/interim/pleiades/Rebull_2016_X_GDR3_supplemented.csv, and not overwrite; returning.
[I 240827 19:19:03 getters:879] Found /mnt_ut3/raid_ut3/home/jerome/github/research/project/ext_tools/gyro-inter

2024-08-27T10:19:06.881348: made ./prot_vs_teff_α_Per_Pleiades_Blanco-1_Psc-Eri_NGC-3532_Group-X_Praesepe_NGC-6811_models_poly7_α_Per_120-Myr_300-Myr_Praesepe_NGC-6811_interpmethodpchip_m67.png
